In [ ]:
import os
import re
import json
import string
import random
import warnings
import requests
import numpy as np 
import pandas as pd 
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
from requests_toolbelt.multipart.encoder import MultipartEncoder
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
warnings.filterwarnings("ignore")

In [ ]:
class GetPartTextFromAudio:
    api_key = 'EB72E803F05E421E93E2'
    
    def __init__(self, wav_file_path):
        self.wav_file_path = wav_file_path
        
    def send_request_for_audio_conversion(self):
        multipart_data = MultipartEncoder(
            fields={
                    'filePath': ('file.wav', open(self.wav_file_path, 'rb')),
                    'samplerate': '16000', 
                    'channel': '1',
                    'normalize': 'yes'
                   }
            )

        response = requests.post('https://apis.sentient.io/microservices/utility/audioprocessing/v0.1/getresults', 
                                 data=multipart_data, 
                                 headers={
                                     'Content-Type': multipart_data.content_type,
                                     'x-api-key': self.api_key
                                 })
        return response.json()


    def send_request_for_speech_to_text_conversion(self, binary_wav_string):
        payload = {
            'model': 'news_parliament',
            'wav_base64': binary_wav_string                                
        }
        payload_json = json.dumps(payload)
        response = requests.post('https://apis.sentient.io/microservices/voice/asr/v0.1/getpredictions', 
                                 data=payload_json, 
                                 headers={
                                     'content-type': 'application/json',
                                     'x-api-key': self.api_key
                                 })
        return response.json()
    
    def get_text_from_response(self, speech_to_text_response):
        response_text = ""
        for speech_utterance in speech_to_text_response['results']:
            response_text = response_text + speech_utterance['text']
        return response_text   
        
    def process(self):
        audio_conversion_response = self.send_request_for_audio_conversion()
        speech_to_text_response = send_request_for_speech_to_text_conversion(audio_conversion_response['results']['AudioContent']) 
        return self.get_text_from_response(speech_to_text_response)

In [ ]:
# GetPartTextFromAudio('resources/music/3.mp3_outputs/spleeter/out001/vocals.wav').process()

In [ ]:
# from rev_ai import apiclient
# client = apiclient.RevAiAPIClient("02TwQ2L4_koragMvDIEReVDmQDwF_Pzoy2JJaPjj1QPaxPEVX-H6lEz4JSh9dYsstFgBt-XZchG4UN8QSmwL3TeD3EM4w")
# job = client.submit_job_local_file('resources/music/1.mp3')
# transcript_json = client.get_transcript_json(job.id)

In [ ]:
# ToDo
# Create a flask script to run as API, get word embeddings-> then cosine similarity and show results

In [ ]:
# def remove_stopword(x):
#     return [y for y in x if y not in stopwords.words('english')]

# def clean_text(text):
#     '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
#     and remove words containing numbers.'''
#     text = str(text).lower()
#     text = text.replace('’','\'')
#     text = re.sub('\[.*?\]', '', text)
#     text = re.sub('https?://\S+|www\.\S+', '', text)
#     text = re.sub('<.*?>+', '', text)
#     text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
#     text = re.sub('\n', '', text)
#     text = re.sub('\w*\d\w*', '', text)
#     text = re.sub('instagram.*', '', text)
#     text = re.sub('piccom.*', '', text)
#     text = re.sub('https|…|twitter|#|@|“|”|°', '', text)
#     text = re.sub('mothers|day|happy|mothersday', '', text)
#     text_list = remove_stopword(str(text).split())    
#     return ' '.join(text_list)

# df['cleaned_lyrics'] = df['seq'].apply(lambda x:clean_text(x))

# df.to_csv('resources/dataset/cleaned.csv')

In [ ]:
# df = pd.read_csv('resources/dataset/cleaned.csv')
# df = df.dropna()
# df = df[['artist', 'song', 'cleaned_lyrics', 'label']]
# df.to_csv('resources/dataset/cleaned.csv')
# df[['artist', 'song']].to_csv('resources/dataset/lookup.csv')
# df.head()

In [ ]:
# module_url = "https://tfhub.dev/google/nnlm-en-dim128/2"
# embed = hub.KerasLayer(module_url)

# arr = np.empty((0,128), int)

# for index, x in  df['cleaned_lyrics'].items():
#     arr = np.append(arr, embed([x]).numpy(), axis=0)

In [ ]:
#  create generate embed 
# getRecommendation -> upload file -> run script -> then sentinet get text -> read csv, generate embed -> cosine -> return list of songs

In [ ]:
# with open('resources/dataset/embedding_lookup.npy', 'wb') as f:
#     np.save(f, arr)

In [ ]:
# with open('resources/dataset/embedding_lookup.npy', 'rb') as f:
#     a1 = np.load(f)

In [ ]:
b = cosine_similarity(arr, a)
c = np.argsort(b, axis=0)[::-1][:4]
c = c.flatten()
c = list(c)
s = df.iloc[c,:]